In [ ]:
"""

Get list of each state's county pages 
from each of the state Wikipedia pages
scraped with code in '01_scrape_wiki_state_list'


output to pickle

"""

In [3]:
from bs4 import BeautifulSoup
import requests

import numpy as np
import pandas as pd

import pickle

import time

In [4]:
# list of state page wikipedia links 

with open('state_wiki_links.pickle', 'rb') as read_file:
    state_links = pickle.load(read_file)
    

In [5]:
st_dc_only = state_links[ :-6]

#drop dc 

st_only = st_dc_only[~st_dc_only['state'].isin(['Washington, D.C.'])].reset_index(drop=True)


In [6]:
st_only.head(5)

,state,wiki_link
0,Alabama,/wiki/List_of_counties_in_Alabama
1,Alaska,/wiki/List_of_boroughs_and_census_areas_in_Alaska
2,Arizona,/wiki/List_of_counties_in_Arizona
3,Arkansas,/wiki/List_of_counties_in_Arkansas
4,California,/wiki/List_of_counties_in_California


In [7]:
url = 'https://en.wikipedia.org'

In [158]:
records=[]

for i,state in enumerate(st_only['state']):
    
    state = st_only['state'][i]
    
    state_page_url = url + st_only['wiki_link'][i]
    
    response = requests.get(state_page_url)
    page = response.text
    soup = BeautifulSoup(page)
    
    table_data = soup.find(class_='wikitable sortable').find_all('th' , scope='row')

    for result in table_data:  
        try:
                        
            FIPS_code = result.findNext('td').text.strip()
            county_name = result.findNext('a').get('title')
            wiki_link = result.findNext('a').get('href') 
            
            records.append((state, FIPS_code, county_name, wiki_link))
       
        except:
            continue
            
    time.sleep(5)


In [160]:
county_links_df = pd.DataFrame(records)

county_links_df.columns = ['state', 'FIPS_code', 'county_name', 'wiki_link']
county_links_df.head(5)

,state,FIPS_code,county_name,wiki_link
0,Alabama,001,"Autauga County, Alabama","/wiki/Autauga_County,_Alabama"
1,Alabama,003,"Baldwin County, Alabama","/wiki/Baldwin_County,_Alabama"
2,Alabama,005,"Barbour County, Alabama","/wiki/Barbour_County,_Alabama"
3,Alabama,007,"Bibb County, Alabama","/wiki/Bibb_County,_Alabama"
4,Alabama,009,"Blount County, Alabama","/wiki/Blount_County,_Alabama"


In [161]:
len(county_links_df)

3095

In [162]:
county_links_df.to_csv('review_county_links.csv')
# reviewed table in CSV

In [164]:
# drop row from Washington state page that is linking to census.gov for info about all states

all_county_pages = county_links_df[~county_links_df['wiki_link'].str.contains('census.gov')].reset_index(drop=True)

In [166]:
len(all_county_pages)

3094

In [167]:
with open('county_wiki_links.pickle', 'wb') as to_write:
    pickle.dump(all_county_pages, to_write)